<center>

## Simulating the Challenger Stage of PGL Antwerp
    
</center>
    
#### Using data from: 
* https://liquipedia.net/counterstrike/PGL/2022/Antwerp
* https://www.hltv.org/
* https://www.hltv.org/news/32621/buchholz-seeding-to-be-used-in-pgl-major-swiss-stages

### Setup

In [1]:
import csgo_sims
from random import choice,randint,uniform
csgo_sims.table.reset()
teams = csgo_sims.teams

#### Load a curated list of tier 1/tier 2 teams (based on HLTV top 30, and my personal opinion)

In [2]:
tiers = {}
with open("tiers.txt","r") as f:
    for rawtier in f.read().split("#")[1:]:
        name = rawtier.split("\n")[0].lower()
        teams = rawtier.split("\n")[1:-1]
        tiers[name] = teams

#### Define function to find recent matches

In [3]:
def find_recents(team):
    out = []
    for match in csgo_sims.recentmatches:
        if match["winner"] == team.name or match["loser"] == team.name:
            out.append(match)
    return out

### Defining a relative "tier" value 
* The tier list is very subjective.
* In order to make this more <i>objective</i>, we consider the number of wins/losses a team has had against different <i>subjective</i> tiers of teams.
* If we use the same <i>subjective</i> tiers, for every team in the tournament, then the calculated "tier" will be more consistent.
* We define different weights for wins against tier 1/tier 2/tier 2 teams, and for losses against tier 1/tier 2/tier 3 teams.
* We completely diregard wins against tier 3 teams, and highly value wins against tier 1 teams.
* Similarly, we heavily punish losses against tier 3 teams, and put less weight on tier 1 losses.

In [4]:
def find_tier(team,tier_weights):
    tier = 0
    data = {"tier1":{"wins":0,"losses":0},"tier2":{"wins":0,"losses":0},"tier3":{"wins":0,"losses":0}}
    for game in find_recents(team):
        if game["winner"] == team.name:
            if game["loser"] in tiers["tier1"]:
                data["tier1"]["wins"] += 1
                tier += tier_weights["tier1"]["win"]
            elif game["loser"] in tiers["tier2"]:
                data["tier2"]["wins"] += 1
                tier += tier_weights["tier2"]["win"]
            else:
                data["tier3"]["wins"] += 1
                tier += tier_weights["tier3"]["win"]
        else:
            if game["winner"] in tiers["tier1"]:
                data["tier1"]["losses"] += 1
                tier -= tier_weights["tier1"]["loss"]
            elif game["winner"] in tiers["tier2"]:
                data["tier2"]["losses"] += 1
                tier -= tier_weights["tier2"]["loss"]
            else:
                data["tier3"]["losses"] += 1
                tier -= tier_weights["tier3"]["loss"]
    return tier,data

### Defining a value for the "form" of a team
* We use the stats "current winstreak" and "win % in the last 3 months" as a measure of the recent form of a team.
* We then normalize this using the "tier" value above (to ensure that lower tier teams with high win % against bad teams dont have a high current form value).

In [5]:
def norm_form(team,
            tier_min = 12,tier_max=40,
            sig_winstreak=5,winstreak_weight=4,last3monthsweight=1,
            tier_weights={"tier1":{"win":3,"loss":1},"tier2":{"win":2,"loss":2},"tier3":{"win":0,"loss":3}}):
    #recent form
    recent_form = (winstreak_weight*(float(team.stats["winstreak"])/sig_winstreak) + last3monthsweight * float(team.stats["win%last3months"]))/100
    relative_tier = (find_tier(team,tier_weights)[0]+tier_min)/tier_max
    normalized_form = recent_form*relative_tier
    return normalized_form

### Defining an "on paper" win/loss probability
* We use the hltv rating and seeds of two teams to estimate the win probability for the first team.

In [6]:
def on_paper(team1,team2,hltv_weight=0.8,seed_weight=0.2):
    total_hltv = team1.stats["hltv"] + team2.stats["hltv"]
    team1_hltv = team1.stats["hltv"]/total_hltv
    team2_hltv = team2.stats["hltv"]/total_hltv

    total_seed = team1.seed + team2.seed
    team1_seed = 1-(team1.seed/total_seed)
    team2_seed = 1-(team2.seed/total_seed)
    team1_total = (team1_seed * seed_weight + team1_hltv * hltv_weight)/(hltv_weight + seed_weight)
    team2_total = (team2_seed * seed_weight + team2_hltv * hltv_weight)/(hltv_weight + seed_weight)
    return team1_total/(team1_total + team2_total)

### Finding historical data
* Look for recent history, and return the record as a fraction.

In [7]:
def get_history(team1,team2):
    wins = 0
    losses = 0
    for match in find_recents(team1):
        if match["winner"] == team1.name and match["loser"] == team2.name:
            wins += 1
        if match["loser"] == team1.name and match["winner"] == team2.name:
            losses += 1
    if (wins + losses) == 0:
        return 0.5
    else:
        return wins/(wins+losses)

### Calculating the "variability" of a team
* Use the "world ranking", "weeks in top 30" and "average player age" as a measure of the experience of a team, and use this to describe how much the performance of a team can fluctuate -> high experience means now variability.

In [8]:
def variabililty(team,worldrankingmax=53,weekstop30max=211,major_effect=2,optimal_age=26):
    rank = (worldrankingmax - team.stats["worldranking"])/worldrankingmax
    weekstop30 = (team.stats["weekstop30"]+major_effect)/(weekstop30max+major_effect)
    age = 1-(abs(team.stats["averageplayerage"]-optimal_age)/optimal_age)
    return (1 - (rank*weekstop30*age))**2

### Calculating weighted the win probability
* Use weighted values of all the parameters above to return a cumulative win probability

In [9]:
def rate(team1,team2,history_weight=0.1,form_weight=3,var_weight=1,upset_weight=0.2):
    team1rate = on_paper(team1,team2)
    team2rate = 1-team1rate
    historyteam1 = get_history(team1,team2)
    historyteam2 = 1-historyteam1
    team1rate += historyteam1*history_weight
    team2rate += historyteam2*history_weight
    total = team1rate + team2rate
    team1rate /= total
    team2rate /= total
    team1form = norm_form(team1)
    team2form = norm_form(team2)
    team1rate += team1form * form_weight
    team2rate += team2form * form_weight
    total = team1rate + team2rate
    team1rate /= total
    team2rate /= total
    team1var = variabililty(team1)
    temp = team1var
    team2var = variabililty(team2)
    team1var = team1var/team2var
    team2var = team2var/temp
    total = team1var + team2var
    team1var = team1var/total
    team2var = team2var/total
    if team1rate > team2rate:
        bad_game_chance = team1var
        good_game_chance = 1-team2var
        upset_chance = bad_game_chance*good_game_chance
        team2rate += upset_chance * upset_weight
    elif team1rate < team2rate:
        bad_game_chance = team2var
        good_game_chance = 1-team1var
        upset_chance = bad_game_chance*good_game_chance
        team1rate += upset_chance * upset_weight
    total = team1rate + team2rate
    team1rate /= total
    team2rate /= total
    return team1rate,team2rate

### Using the probability to simulate a game
* This is a very simple simulation, that just uses the probability to decide who wins each round.

In [10]:
def simfunc(team1,team2):
    team1win,team2win = rate(team1,team2)
    bo3 = csgo_sims.isbo3(team1,team2)
    games_to_win = 1
    if bo3:
        games_to_win = 2
    team1games = 0
    team2games = 0
    team1totalrounds = 0
    team2totalrounds = 0
    while team1games != games_to_win and team2games != games_to_win:
        team1rounds = 0
        team2rounds = 0
        while team1rounds != 16 and team2rounds != 16:
            if uniform(0,1) < team1win:
                team1rounds += 1
            else:
                team2rounds += 1
        team1totalrounds += team1rounds
        team2totalrounds += team2rounds
        if team1rounds > team2rounds:
            team1games += 1
        else:
            team2games += 1
    if team1games > team2games:
        return team1,csgo_sims.GameStats(team1totalrounds-team2totalrounds,(team1games,team2games))
    else:
        return team2,csgo_sims.GameStats(team2totalrounds-team1totalrounds,(team2games,team1games))

### Running 1000 simulations to get promotion probability

In [13]:
results = csgo_sims.table.do_sims(simfunc,nsims=1000)

Team             Promotion% (3-0)% (3-1)% (3-2)% (2-3)% (1-3)% (0-3)%
G2                     100%    79%    19%     2%     0%     0%     0%
ENCE                   100%    99%     1%     0%     0%     0%     0%
Astralis                99%    16%    64%    19%     1%     0%     0%
Vitality                97%     6%    62%    29%     2%     1%     0%
Outsiders               91%     0%    47%    44%     8%     1%     0%
Liquid                  90%     0%    42%    47%     8%     2%     0%
MIBR                    84%     0%    36%    48%    14%     2%     0%
Spirit                  54%     0%    15%    39%    35%    10%     1%
Imperial                28%     0%     4%    24%    45%    27%     1%
Bad News Eagles         26%     0%     5%    22%    50%    23%     1%
Complexity              16%     0%     1%    15%    43%    39%     3%
Eternal Fire             9%     0%     2%     7%    38%    49%     5%
forZe                    7%     0%     1%     6%    46%    46%     2%
Renegades           

### Running just one simulation

In [14]:
results = csgo_sims.table.play(simfunc)

============================Round0===========================
Vitality        (1-0) beats Complexity       (0-1), 1-0 (+3)
Bad News Eagles (1-0) beats Eternal Fire     (0-1), 1-0 (+2)
G2              (1-0) beats Liquid           (0-1), 1-0 (+4)
Renegades       (1-0) beats forZe            (0-1), 1-0 (+3)
Astralis        (1-0) beats IHC              (0-1), 1-0 (+14)
Spirit          (1-0) beats Imperial         (0-1), 1-0 (+5)
MIBR            (1-0) beats Outsiders        (0-1), 1-0 (+3)
ENCE            (1-0) beats 9z               (0-1), 1-0 (+14)
============================Round1===========================
ENCE            (2-0) beats Renegades        (1-1), 1-0 (+13)
G2              (2-0) beats Spirit           (1-1), 1-0 (+9)
Astralis        (2-0) beats Bad News Eagles  (1-1), 1-0 (+11)
Vitality        (2-0) beats MIBR             (1-1), 1-0 (+2)
forZe           (1-1) beats 9z               (0-2), 1-0 (+8)
Liquid          (1-1) beats Imperial         (0-2), 1-0 (+3)
Eternal Fire    (1